In [15]:
import abc

def test_has_choice_with_length_gt_1(choices):
    return max([len(x) for x in choices]) > 1

def print_choices_with_number(choices):
    for i, x in enumerate(choices):
        print("{0}. {1}".format(i, x), flush=True)


class AskStrategy(abc.ABC):
    @abc.abstractmethod
    def ask_for_int(): pass

    @abc.abstractmethod
    def ask_for_char(): pass


def ask_for_int():
    return int(input())
def ask_for_char():
    return input()

class StdinAskStrategy(AskStrategy):
    def ask_for_int(self):
        return ask_for_int()
    def ask_for_char(self):
        return ask_for_char()


class Ask():
    def __init__(self, choices=['y', 'n'], ask_strategy=StdinAskStrategy()):
        self.choices = choices
        self.ask_strategy = ask_strategy
    def ask(self):
        if test_has_choice_with_length_gt_1(self.choices):
            print_choices_with_number(self.choices)
            ask_res = self.ask_strategy.ask_for_int()
            return self.choices[ask_res]
        else:
            print("/".join(self.choices), flush=True)
            return self.ask_strategy.ask_for_char()


class Content():
    def __init__(self,x):
        self.x = x

class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

In [8]:
import unittest

# class MockAskStrategy(AskStrategy):
#     def ask_for_char():
#         return ()

class TestAsk(unittest.TestCase):
    def test_has_choice_with_length_gt_1(self):
        self.assertEqual(test_has_choice_with_length_gt_1(['abc', 'def']), True)

    def test_has_choice_with_length_gt_1_false(self):
        self.assertEqual(test_has_choice_with_length_gt_1(['y', 'n']), False)

unittest.main(argv=[''], verbosity=2, exit=False)

test_has_choice_with_length_gt_1 (__main__.TestAsk) ... ok
test_has_choice_with_length_gt_1_false (__main__.TestAsk) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


In [16]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivore': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lays eggs'])])),
    'animal:monkey' : If(['mammal','carnivore','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivore','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long neck','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

In [17]:
def get_from_memory(name, memory):
    found = (name in memory)
    return (found,  memory[name] if found else None)

def get_AND_of_exprs(expr, eval):
    ## get list out
    expr_list = expr.x if isinstance(expr,AND) else expr
    all_y = all(eval(x) == 'y' for x in expr_list)
    return 'y' if all_y else 'n'

def get_OR_of_exprs(expr, eval):
    ## get list out
    expr_list = expr.x
    any_y = any(eval(x) == 'y' for x in expr_list)
    return 'y' if any_y else 'n'

def get_rule_true_value(name, rule_attr):
    # attr:expected_value expresses that we want to (1) get the value of `attr` and compare it to `expected_value`
    # Returns `y` when they are equal, else `n`
    return 'y' if rule_attr==name else rule_attr.split(':')[1]


class KnowledgeBase():
    def __init__(self, rules):
        self.rules = rules
        self.memory = {}

    def get(self, rule_attr_in):
        if ':' in rule_attr_in:
            rule_A, rule_V = rule_attr_in.split(':')
            eval_V = self.get(rule_A) # descend
            # check that the `value`` from attr:`value` is equal to the eval'd value
            return 'y' if rule_V==eval_V else 'n'

        # Memory
        found,in_memory_result = get_from_memory(rule_attr_in, self.memory)
        if found: return in_memory_result

        # Try each rule
        for rule_A in self.rules.keys():
            if rule_A==rule_attr_in or rule_A.startswith(rule_attr_in+":"):
                value_when_true = get_rule_true_value(rule_attr_in, rule_A)
                eval_res = self.eval(self.rules[rule_A], field=rule_attr_in)

                if eval_res != 'y' and eval_res != 'n' and value_when_true =='y':
                    self.memory[rule_attr_in] = eval_res
                    return eval_res
                if eval_res == 'y':
                    self.memory[rule_attr_in] = value_when_true
                    return value_when_true

        # field is not found, using default
        res = self.eval(self.rules['default'],field=rule_attr_in)
        self.memory[rule_attr_in]=res
        return res


    def eval(self, expr, field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr, Ask): # ASK
            print(field)
            return expr.ask()
        elif isinstance(expr, If): # IF
            return self.eval(expr.x)
        elif isinstance(expr, AND) or isinstance(expr, list):
            return get_AND_of_exprs(expr, self.eval)
        elif isinstance(expr,OR):
            return get_OR_of_exprs(expr, self.eval)
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

In [11]:
import unittest

class TestAsk(unittest.TestCase):
    def test_get_from_memory(self):
        self.assertEqual(get_from_memory("x", {"x": "Found it!"}), (True, "Found it!"))
    def test_get_from_memory_not_found(self):
        self.assertEqual(get_from_memory("x", {"abc": "Found it!"}), (False, None))
    def test_get_AND_of_exprs(self):
        self.assertEqual(get_AND_of_exprs(['y', 'y', 'y'], lambda x: x), 'y')
    def test_get_AND_of_exprs_n(self):
        self.assertEqual(get_AND_of_exprs(['y', 'n', 'y'], lambda x: x), 'n')
    def test_get_OR_of_exprs(self):
        self.assertEqual(get_OR_of_exprs(OR(['y', 'n', 'n']), lambda x: x), 'y')
    def test_get_OR_of_exprs_n(self):
        self.assertEqual(get_OR_of_exprs(OR(['n', 'n', 'n']), lambda x: x), 'n')


unittest.main(argv=[''], verbosity=2, exit=False)

test_get_AND_of_exprs (__main__.TestAsk) ... ok
test_get_AND_of_exprs_n (__main__.TestAsk) ... ok
test_get_OR_of_exprs (__main__.TestAsk) ... ok
test_get_OR_of_exprs_n (__main__.TestAsk) ... ok
test_get_from_memory (__main__.TestAsk) ... ok
test_get_from_memory_not_found (__main__.TestAsk) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.011s

OK


In [12]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
eats meat
y/n
carnivor
y/n
has hooves
y/n
chews cud
y/n
ungulate
y/n
feathers
y/n
flies
y/n
lies eggs
y/n
long nech
y/n
color
0. red-brown
1. black and white
2. other
swims
y/n
flies well
y/n


'albatross'

In [2]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to c:\users\carlk\appdata\local\temp\pip-req-build-b7b65c91

  Running command git clone -q https://github.com/buguroo/pyknow/ 'C:\Users\carlk\AppData\Local\Temp\pip-req-build-b7b65c91'



  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34607 sha256=986f570449c46b532c51451bf38c2af1df3b910a95826a7475bc72635826e31f
  Stored in directory: C:\Users\carlk\AppData\Local\Temp\pip-ephem-wheel-cache-vu_9jsq3\wheels\83\1a\5a\662134cfe4e4c48dc95286cb484685e55ab4c98f4ada37dfe7
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3184 sha256=8265a6f7165dcc3715887eca28d37ea2f6b1785db80a54379a8caa81cd8d9dad
  Stored in directory: c:\users\carlk\appdata\local\pip\cache\wheels\9b\9b\56\5713233cf7226423ab6c58c08081551a301b5863e343ba053c
Successfully built pyknow frozendict


In [5]:
from pyknow import *
#import pyknow

In [12]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def carnivore(self):
        self.declare(Fact('carnivore'))

    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')

    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')

    @Rule(Fact('mammal'),Fact('carnivore'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivore'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='penguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))

    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))

    def factz(self,l):
        for x in l:
            self.declare(x)

In [13]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivore')),
          (9, Fact(animal='tiger'))])